In [1]:
from dhanhq import dhanhq, marketfeed
import talib
import numpy as np
import pandas as pd
import time
import datetime
import os

In [2]:
with open("token.txt", "r") as file:
    # Read the entire contents of the file
    token = file.read()
client_id = "1102249582"
access_token = token
dhan = dhanhq(client_id,access_token)

masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', nrows=1)
masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', usecols=masterlist.columns, low_memory=False)




In [11]:
BANK_NIFTY_PREMIMUM = 53000

filtered_ce_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('BANKNIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2024-10-01 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == BANK_NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'CE') 
]
filtered_ce_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
128507,NSE,D,59178,OPTIDX,0,BANKNIFTY-Oct2024-53000-CE,15.0,BANKNIFTY 01 OCT 53000 CALL,2024-10-01 14:30:00,53000.0,CE,5.0,W,OP,NaN,NaN


In [12]:
filtered_pe_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('BANKNIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2024-10-01 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == BANK_NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'PE')
]

filtered_pe_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
128508,NSE,D,59179,OPTIDX,0,BANKNIFTY-Oct2024-53000-PE,15.0,BANKNIFTY 01 OCT 53000 PUT,2024-10-01 14:30:00,53000.0,PE,5.0,W,OP,NaN,NaN


In [ ]:

SECURITY_ID = 59179

IS_ACTIVE = False
IS_OPEN_POSITION = False
quantity = 0
buy_price = 0     
stop_loss = 0

while IS_ACTIVE is False:
        current_second = datetime.datetime.now().second
        time.sleep(1)
        print("waiting to activate", current_second)
        if current_second == 1:
            IS_ACTIVE = True

while IS_ACTIVE is True and IS_OPEN_POSITION is False:
      current_second = datetime.datetime.now().second
      if current_second == 0:
            break;
      print('Active',current_second)
      time.sleep(0.15)
      intraday = dhan.intraday_daily_minute_charts(SECURITY_ID, dhan.NSE_FNO, "OPTIDX")
      print("intraday", intraday)
      intraday_df = pd.DataFrame(intraday['data']).tail(2)
      temp_list = []

      for i in intraday_df['start_Time']:
          temp = dhan.convert_to_date_time(i)
          temp_list.append(temp)

      intraday_df['Date'] = temp_list

      previous_closed_candle = intraday_df.iloc[-2]
      current_running_candle = intraday_df.iloc[-1]
      
      if current_running_candle['close'] > previous_closed_candle['high']:
            print('Buy')
            length_of_previous_candle = round(previous_closed_candle['high'] - previous_closed_candle['low'])
            risk                      = 1000
            quantity                  = round(round(risk / length_of_previous_candle) / 15) * 15
            print('quantity',quantity)
            
            if quantity > 300:
                  quantity = 300

            buy_price = int(current_running_candle['close'])
            stop_loss = int(previous_closed_candle['low'])

            dhan.place_order(
            security_id=SECURITY_ID,
            exchange_segment=dhan.NSE_FNO,
            transaction_type=dhan.BUY,
            quantity=quantity,
            order_type=dhan.MARKET,
            product_type=dhan.INTRA,
            price=0)  

            IS_OPEN_POSITION = True
            break;




In [ ]:

COST_TO_COST_SL = False
TRAILING_SL = True

while IS_OPEN_POSITION is True and quantity > 0:
      time.sleep(0.10)
      print("stop_loss :-", stop_loss)
      intraday = dhan.intraday_daily_minute_charts(SECURITY_ID, dhan.NSE_FNO, "OPTIDX")
      print("waiting for sell")
      intraday_df = pd.DataFrame(intraday['data']).tail(2)
      temp_list = []

      for i in intraday_df['start_Time']:
          temp = dhan.convert_to_date_time(i)
          temp_list.append(temp)

      intraday_df['Date'] = temp_list

      previous_closed_candle = intraday_df.iloc[-2]
      current_running_candle = intraday_df.iloc[-1]

      # Stop loss
      if current_running_candle['close'] < stop_loss and COST_TO_COST_SL is False and TRAILING_SL is False:
            print("SELL")
            IS_OPEN_POSITION = False
          
           # Place an order for NSE Futures & Options
            dhan.place_order(security_id=SECURITY_ID,
            exchange_segment=dhan.NSE_FNO,
            transaction_type=dhan.SELL,
            quantity=quantity,
            order_type=dhan.MARKET,
            product_type=dhan.INTRA,
            price=0)  

      # Trailing SL
      if current_running_candle['close'] < previous_closed_candle['low'] and COST_TO_COST_SL is False and TRAILING_SL is True:
            print("SELL")
            IS_OPEN_POSITION = False
          
           # Place an order for NSE Futures & Options
            dhan.place_order(security_id=SECURITY_ID,
            exchange_segment=dhan.NSE_FNO,
            transaction_type=dhan.SELL,
            quantity=quantity,
            order_type=dhan.MARKET,
            product_type=dhan.INTRA,
            price=0)  

      # Shift sl to Cost to Cost
      if current_running_candle['close'] < buy_price +1 and COST_TO_COST_SL is True and TRAILING_SL is False:
            print("SELL")
            IS_OPEN_POSITION = False
          
           # Place an order for NSE Futures & Options
            dhan.place_order(security_id=SECURITY_ID,
            exchange_segment=dhan.NSE_FNO,
            transaction_type=dhan.SELL,
            quantity=quantity,
            order_type=dhan.MARKET,
            product_type=dhan.INTRA,
            price=0)  

            